In [21]:
# import librosa
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from IPython.display import Audio
import collections
import numpy as np
from microphone import record_audio
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.filters import minimum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion 
from scipy.ndimage.morphology import iterate_structure
from collections import defaultdict
%matplotlib notebook
import librosa


In [22]:
def loadsong(path):
    songname, fs = librosa.load(path, sr=44100, mono=True)
    spec, freqs, times = mlab.specgram(songname, NFFT = 4096, Fs = fs, window=mlab.window_hanning, noverlap=(4096//2))
    return spec, freqs, fs

In [23]:

S, freqs, fs = loadsong(r"/Users/ji-macbook15/Desktop/moo/1.mp3")

S2, freqs2, fs2 = loadsong(r"/Users/ji-macbook15/Desktop/moo/2.mp3")

S3, freqs3, fs3 = loadsong(r"/Users/ji-macbook15/Desktop/moo/3.mp3")

S4, freqs4, fs4 = loadsong(r"/Users/ji-macbook15/Desktop/moo/4.mp3")

S5, freqs5, fs5 = loadsong(r"/Users/ji-macbook15/Desktop/moo/5.mp3")

S6, freqs6, fs6 = loadsong(r"/Users/ji-macbook15/Desktop/moo/6.mp3")


In [26]:
def findpeaks(spec):
    ys, xs = np.histogram(spec.flatten(), bins=spec.size//2, normed=True)
    dx = xs[-1] - xs[-2]
    cdf = np.cumsum(ys)*dx  # this gives you the cumulative distribution of amplitudes
    cutoff = xs[np.searchsorted(cdf, 0.77)]

    foreground = (spec >= cutoff)
    
    struct = generate_binary_structure(2,1)
    neighborhood = iterate_structure(struct, 20)
    local = spec == maximum_filter(spec, footprint=neighborhood)
    
    intersection = local & foreground
    bins = np.argwhere(intersection) 
    return bins

In [24]:
lyst = list(range(10))
lyst[99:100]

[]

In [27]:
database = collections.defaultdict(list)
songnames = []

def addtodb(name, artist, bins):
    songnames.append(name)
    for index, (f1, t1) in enumerate(bins):
        
        fanout = bins[index+1:index+21]   
        
        for f2, t2 in fanout:
            x = (f1, f2, t2-t1)
            database[x].append((name, artist, t1)) # if we have a key for this fp, we add the song,artist, and time at which first peak occurred 

addtodb("what u need", "the weeknd", findpeaks(S2))
addtodb("im the one", "dj khaled ft. justin bieber, quavo, chance the rapper, lil wayne", findpeaks(S))
addtodb("in the night", "the weeknd", findpeaks(S3))
addtodb("work from home", "5h", findpeaks(S4))
addtodb("night changes", "1d", findpeaks(S5))
addtodb("blank space", "taylor swift", findpeaks(S6))

In [64]:
#lol = [(('in the night', 'the weeknd', 3130, 2), 2), (('in the night', 'the weeknd', 339, 2), 1), (('night changes', '1d', 1083, 2), 1), (('in the night', 'the weeknd', 362, 2), 1), (('in the night', 'the weeknd', 2346, 2), 1), (('in the night', 'the weeknd', 85, 2), 1), (('in the night', 'the weeknd', 108, 2), 1), (('in the night', 'the weeknd', 131, 2), 1), (('in the night', 'the weeknd', 154, 2), 1), (('in the night', 'the weeknd', 177, 2), 1), (('in the night', 'the weeknd', 200, 2), 1), (('in the night', 'the weeknd', 454, 2), 1), (('in the night', 'the weeknd', 1423, 2), 1), (('in the night', 'the weeknd', 2023, 2), 1), (('in the night', 'the weeknd', 2046, 2), 1), (('in the night', 'the weeknd', 2069, 2), 1), (('in the night', 'the weeknd', 3107, 2), 1), (('in the night', 'the weeknd', 3361, 2), 1), (('in the night', 'the weeknd', 3453, 2), 1), (('in the night', 'the weeknd', 4445, 2), 1), (('in the night', 'the weeknd', 4745, 2), 1)]


def findprob(song):
    songcounts = {}
    count = 0
    probs = ""
    
    for asong in song:
        if asong[0][0] in songcounts:
            songcounts[asong[0][0]] += asong[1]
        else:
            songcounts[asong[0][0]] = asong[1]
        count += asong[1]
        
    if count < 10:
        return "can not find any songs"
    
    for i in range(len(songnames)):
        try:
            probs += str(np.round(songcounts[songnames[i]] / count, decimals = 3) * 100)
            probs += "% chance of being "
            probs += songnames[i]
            probs += ". "
        except:
            pass
    return probs


In [65]:
def match_song(db, excerpt): #note to self: excerpt = [(t1,f1),(t2,f2), etc]
    templist = [] 
    
    for index, (fe1, te1) in enumerate(excerpt):
        fanout = excerpt[index+1:index+21]
        
        for fe2, te2 in fanout:
            x = (fe1, fe2, te2-te1)
            
            values = db.get(x)
            
            if values is None:
                continue
            else:
                for avalue in values:
                    templist.append((avalue[0], avalue[1], avalue[2] - te1, 2))
    
    return findprob(collections.Counter(templist).most_common()) 

def make_excerpt(time):
    recording = record_audio(time)
    excerpt = np.hstack([np.fromstring(i, np.int16) for i in recording[0]])

    excerpt_spec, freqs_spec, times_spec = mlab.specgram(excerpt, NFFT=4096, Fs=fs,
                                                      window=mlab.window_hanning,
                                                      noverlap=(4096 // 2)) 

    return findpeaks(excerpt_spec)



In [72]:

match_song(database, make_excerpt(30))


No microphone configuration file found, attempting to find default device..
Using input device 'Built-in Microphone'
Recording ended


'0.3% chance of being in the night. 0.5% chance of being work from home. 99.2% chance of being blank space. '


    # for notes in excerpt: #look at every tuple (f1,f2,deltaT) within the song excerpt
     #   if notes in database: #if such fp is present in database
      #      for value in database[notes]: # go thru every value associated with the key
       #         print(value[2])
        #        print(notes[2])
         #       templist.append((value[0], value[1], value[2] - notes[2]))   
          #      #we add the song name(value[0]), the artists (value[1]), and the offset in time between when the first 
           #     #peak appears in the database - first peak appears in the song excerpt
            #    
             #   #IGNORE: templist.append(tuple(database[notes])) #dont forget to do the more advanced portion
    #counts = collections.Counter(templist) 

    #if (len(counts) > 0): #if we find any matches
     #   print(counts.most_common())
      #  return counts.most_common()[0][0][0] + " by " + counts.most_common()[0][0][1]
    #else: